<a href="https://colab.research.google.com/github/nreichen/eecs448-CollegeSubredditsCommentGeneration/blob/main/subredditTrainedModelsNotebooks/Ohio_Final_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os 
GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE = "448 Group: Jedi Order/models"
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['GoogleNews-vectors-negative300.bin.gz', 'COPY!!!!!!!!!!!!!!!!!.ipynb', 'gpt2-gerchef', 'Copy of Fine-tune a non-English GPT-2 Model with Huggingface', 'tess_copy_commentgeneration.ipynb', 'LifeProTips', 'UIUC', 'bigtenLinearSVC.pickle', 'model1.ipynb', 'UIUC2', 'preprocessCornell.ipynb', 'uofm', 'askredditpreprocessing.ipynb', 'gpt-2-with-huggingface-pytorch.ipynb', 'Test_Evaluation_Metrics.ipynb.ipynb', 'commentgeneration.ipynb', 'train_post_set.pickle', 'test_post_set.pickle', 'train_com_set.pickle', 'test_com_set.pickle', 'todayilearnedgeneration.ipynb', 'val_post_set.pickle', 'val_com_set.pickle', 'COPY FOR TRYING SEQTOSEQ uiucgenerationmanual.ipynb', 'Cornell Final Train.ipynb', 'Illinois', 'UMD', 'Indiana', 'Iowa', 'msu', 'uofmn', 'Penn', 'Ohio', 'Penn Final Train.ipynb', 'uofmn Final Train.ipynb', 'lincoln', 'Northwestern', 'rutgers', 'UWMadison', 'Cornell (OLD FOLDER)', 'Cornell', 'Purdue', 'Iowa Train Final.ipynb', 'Ohio Final Train.ipynb', 'Illinoi

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(GOOGLE_DRIVE_PATH)

In [ ]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install transformers
!pip install convokit
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.0/168.0 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.6 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-2.5.3-py3-none-any.whl size=

In [ ]:
import pickle
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, EarlyStoppingCallback
from datasets import load_dataset
import nltk
# maybe modify the above to a different model? idk up to you all
# also how to use the gpu command? like the cuda stuff.
# 
from convokit import Corpus, download
from torch.utils.data import DataLoader, IterableDataset, Dataset
import torch
import numpy as np

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Load and Prepare Dataset


In [ ]:
seed = 448
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [ ]:
GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_DATA = "448 Group: Jedi Order/data/CornellSubreddits/OSU.corpus"
DATA_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_DATA)
# corpus = Corpus(filename=download("subreddit-Cornell"))
corpus = Corpus(filename=DATA_PATH)
corpus.print_summary_stats()

Number of Speakers: 18298
Number of Utterances: 289700
Number of Conversations: 32115


In [ ]:
convo_df = corpus.get_conversations_dataframe()
utt_df = corpus.get_utterances_dataframe()
utt_ids = corpus.get_utterance_ids()

In [ ]:
temp = utt_df.sort_values("meta.score", ascending=False)
temp.head()

,timestamp,text,speaker,reply_to,conversation_id,meta.score,meta.top_level_comment,meta.retrieved_on,meta.gilded,meta.gildings,meta.subreddit,meta.stickied,meta.permalink,meta.author_flair_text,vectors
id,,,,,,,,,,,,,,,
4cv7zy,1459511301,,MJGUHD,None,4cv7zy,921,None,1463512751,0,None,OSU,False,/r/OSU/comments/4cv7zy/throwback_to_cookiezis_...,,[]
5fc200,1480345019,"According to a relative who works at OSU, ther...",XxgobuckeyesxX,None,5fc200,887,None,1484282404,0,None,OSU,False,/r/OSU/comments/5fc200/shooter_in_macquigg/,,[]
9k134u,1538259563,,nordicskier17,None,9k134u,522,None,1540193794,0,"{'gid_1': 0, 'gid_2': 0, 'gid_3': 0}",OSU,False,/r/OSU/comments/9k134u/took_this_pic_last_nigh...,,[]
9hvd6v,1537574661,,HungoverSundays,None,9hvd6v,517,None,1540157522,0,"{'gid_1': 0, 'gid_2': 0, 'gid_3': 0}",OSU,False,/r/OSU/comments/9hvd6v/youre_god_damn_right/,,[]
9b0ocp,1535475710,,darthgallion,None,9b0ocp,515,None,1536819222,0,None,OSU,False,/r/OSU/comments/9b0ocp/me_as_an_alum_looking_a...,"Civil Engineering, 17",[]


In [ ]:
temp = utt_df.sort_values("meta.score", ascending=False)

posts = temp[temp['reply_to'].isnull()]
comments = temp[temp['reply_to'].notnull()]

posts = posts[(posts["text"] != '') & (posts["text"] != '[deleted]')  & (posts["text"] != '[removed]') & (posts["text"] != ' ')]
comments = comments[(comments["text"] != '') & (comments["text"] != '[deleted]')  & (comments["text"] != '[removed]') & (comments["text"] != ' ')]

conversation_ids_in = list(posts["conversation_id"])

coversation_ids = []
num_ids = 0
for id in conversation_ids_in:
  if num_ids == 20000:
      break
  dataframe = comments[comments["conversation_id"] == id]
  com_ids = list(dataframe.index)
  if com_ids != []:
    coversation_ids.append(id)
    num_ids = num_ids + 1

top_posts = posts.head(20000)
top_posts.head()

print(len(top_posts))


20000


In [ ]:
coversation_ids = coversation_ids[:20000] #not sure if needed 

In [ ]:
datasets_to_use = []
train_dataset_to_use = []
val_dataset_to_use = []
test_dataset_to_use = []
train_com = []
val_com = []
test_com = []
train_len = []
val_len = []
test_len = []
train_com_len = []
val_com_len = []
test_com_len = []

In [ ]:
import re
from nltk import word_tokenize
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from sklearn.model_selection import train_test_split

tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token = "<bos>", eos_token ="<eos>", truncation_side='right')
train_ids, test_ids = train_test_split(coversation_ids, test_size=0.25, random_state=42)
val_ids, test_ids = train_test_split(test_ids, test_size=0.5, random_state=42)
ids_total = [train_ids, val_ids, test_ids]
post_len = [train_len, val_len, test_len]
com_len = [train_com_len, val_com_len, test_com_len]
posts_datasets_to_use = [train_dataset_to_use, val_dataset_to_use, test_dataset_to_use]
comments_datasets_to_use = [train_com, val_com, test_com]
for index, id_list in enumerate(ids_total):
  for id in id_list:
      dataframe = comments[comments["conversation_id"] == id]
      post = posts.loc[id]['text']
      post = re.sub("\n", " ", post)
      post = re.sub("&gt;", "", post)
      post = re.sub("  ", " ", post)
      post = re.sub("&amp;", "", post)
      post = re.sub("#x200B;", "", post)
      post = re.sub(r'http\S+', '[url link]', post)
      post_tokens = word_tokenize(post)
      len_post = len(post_tokens)
      post_len[index].append(len_post)
      commented_here = ""
      com_ids = list(dataframe.index)
      if (len_post <= 512):
          running_sum = 0 # testing
          for id_c in com_ids:
            out_text = dataframe.loc[id_c]['text']
            out_text = str(out_text).strip()
            out_text = re.sub(r"\s", " ", out_text)
            out_text = re.sub(r"\n", " ", out_text)
            out_text = re.sub("&gt;", "", out_text) # added preprocess
            out_text = re.sub("  ", " ", out_text) # added preprocess
            out_text = re.sub("&amp;", "", out_text) # added preprocess
            out_text = re.sub("#x200B;", "", out_text) # added preprocess
            out_text = re.sub(r'http\S+', '[url link]', out_text) # added preprocess
            out_text_tokens = word_tokenize(out_text) # added this
            running_sum += len(out_text_tokens) # added this
            commented_here = commented_here + " " + out_text
            if running_sum > len_post: # changed this to 512
              break # added this
          comment_tokens = word_tokenize(commented_here)
          com_len[index].append(len(comment_tokens))
          post_utter = "<bos> " + post + " <eos> "
          comment_utter = "<bos>" + commented_here + " <eos> "
          posts_datasets_to_use[index].append(post_utter)
          comments_datasets_to_use[index].append(comment_utter)
print(posts_datasets_to_use[0][:10])
print(comments_datasets_to_use[0][:10])
print(len(posts_datasets_to_use[0]))
print(len(comments_datasets_to_use[0]))
print(post_len[0][0])
print(com_len[0][0])
print(len(post_len[0]))
print(len(com_len[0]))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


["<bos> I really need to get into Criticizing tv, and it fills up so fast it's disgusting.  <eos> ", "<bos> Hello. Due to one of my transfer courses being approved(when they said it wouldn't), I have a 3 Credit hour slot free for Spring 18. I'm looking for a Class that fills the ORGANIZATIONS AND POLITIES, and the HUMAN, NATURAL, AND ECONOMIC RESOURCES categories in the Social Science part. I know that almost all the appealing classes might already been taken, but I can't just let these credit hours go. So, I appreciate any suggestions you guys might have, <eos> ", "<bos> Hey, so in order to get the extra ticket for the next day for free, do we need to stay at one of Cedar Point's hotels or any hotel? Thanks! <eos> ", '<bos> I am going on my second year and I am number 13 on the wait list for ENGR 1182. Should I just go ahead and schedule into another one or wait a few more weeks? <eos> ', "<bos> I was forced to drop a class, due to the fact that it didn't fulfill the requirement I tho

In [ ]:
train_tensors = []
val_tensors = []
test_tensors = []
train_com_tensors = []
val_com_tensors = []
test_com_tensors = []
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
print(len(tokenizer))

50260


In [ ]:
post_tensors_data = [train_tensors, val_tensors, test_tensors]
comments_tensors_data = [train_com_tensors, val_com_tensors, test_com_tensors]
# there doesn't seem to be a way to associate the two permanently, since tokenizer automatically removes it.
# maybe the exception is you reach into the other comment and remove?
# for instance, loop with like index: if it excepts, then: you go into the other coms dataset_to_use and remove that specific index.
# say in train, which is index = 0, and we find that at index1, post is too long. go into coms_dataset_to_use and delete that entry.
# i assume that it does the same thing for the other one. 
post_max = []
com_max = []
for index, dataset_tensors in enumerate(posts_datasets_to_use):
  for index1, text in enumerate(dataset_tensors):
    post_tensors_data[index].append(text)
    #except:
    #print(index1) # watch for big tokens how to. like does this already truncate? 
    #del comments_datasets_to_use[index][index1]
    #del posts_datasets_to_use[index][index1] #maybe this one too for the next pass around
      # only problem is are the indexes reliable? like does index1 mean the same thing in both when you delete
      # and does a failed tokenizer actually do anything

for index, dataset_tensors in enumerate(comments_datasets_to_use):
  for index1, text in enumerate(dataset_tensors):
    comments_tensors_data[index].append(text)

#for index, dataset_tensors in enumerate(posts_datasets_to_use):
#  for index1, text in enumerate(dataset_tensors):
#    encoded_input = tokenizer(text, return_tensors="pt", padding=True)
#    tensors_data[index].append(encoded_input)

In [ ]:
print(post_tensors_data[0][0])

<bos> I really need to get into Criticizing tv, and it fills up so fast it's disgusting.  <eos> 


In [ ]:

GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_INFO = "448 Group: Jedi Order/models/Ohio"
PATH_TO_PUT = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_INFO)
print(GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_INFO)
print(os.listdir(PATH_TO_PUT))

448 Group: Jedi Order/models/Ohio
['train_post_set.pickle', 'val_post_set.pickle', 'test_post_set.pickle', 'train_com_set.pickle', 'val_com_set.pickle', 'test_com_set.pickle', 'checkpoints-Ohio']


In [ ]:
train_post_path = os.path.join(PATH_TO_PUT, "train_post_set.pickle")
val_post_path = os.path.join(PATH_TO_PUT, "val_post_set.pickle")
test_post_path = os.path.join(PATH_TO_PUT, "test_post_set.pickle")
train_com_path = os.path.join(PATH_TO_PUT, "train_com_set.pickle")
val_com_path = os.path.join(PATH_TO_PUT, "val_com_set.pickle")
test_com_path = os.path.join(PATH_TO_PUT, "test_com_set.pickle")

pickle.dump(posts_datasets_to_use[0], open(train_post_path, "wb"))
pickle.dump(posts_datasets_to_use[1], open(val_post_path, "wb"))
pickle.dump(posts_datasets_to_use[2], open(test_post_path, "wb"))
pickle.dump(comments_datasets_to_use[0], open(train_com_path, "wb"))
pickle.dump(comments_datasets_to_use[1], open(val_com_path, "wb"))
pickle.dump(comments_datasets_to_use[2], open(test_com_path, "wb"))

In [ ]:
posts_datasets_to_use[0] = pickle.load(open(train_post_path, "rb"))
posts_datasets_to_use[1] = pickle.load(open(val_post_path, "rb"))
posts_datasets_to_use[2] = pickle.load(open(test_post_path, "rb"))
comments_datasets_to_use[0] = pickle.load(open(train_com_path, "rb"))
comments_datasets_to_use[1] = pickle.load(open(val_com_path, "rb"))
comments_datasets_to_use[2] = pickle.load(open(test_com_path, "rb"))

In [ ]:
import tensorflow
### dataset.py ###
class RedditDataset(Dataset):
  """Dataset for our Reddit thing"""

  def __init__(self, posts, comments, dataset_type: str = 'Train'):
    super().__init__()
    self.posts = posts
    self.comments = comments
    self.dataset_type = dataset_type
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.posts)

  def get_labels(self) -> pd.Series:
    return self.comments
  
  def __getitem__(self, idx):
    post_encoding = self.posts[idx]
    comment_encoding = self.comments[idx]
    if self.dataset_type == 'Train':
      token = self.tokenizer.encode(post_encoding + 'TL;DR' + comment_encoding, return_tensors="pt", 
                                    padding="max_length", truncation=True)
    return token

In [ ]:
print(post_tensors_data[0][0])
print(comments_tensors_data[0][0])

<bos> I really need to get into Criticizing tv, and it fills up so fast it's disgusting.  <eos> 
<bos> Well this was helpful don't take my spot slut. Haha fun fact I have a criticizing TV midterm essay due tomorrow that's 4-6 pages and I just started <eos> 


In [ ]:
#print(len(tensors_data[index][0]["input_ids"][0]))
#print(len(tensors_data[index][0]["attention_mask"][0]))
#print(encoded_input["input_ids"].shape[1])
#print(encoded_input["attention_mask"].shape[1])

#train_to = zip(post_tensors_data[0], comments_tensors_data[0])
train_data = RedditDataset(post_tensors_data[0], comments_tensors_data[0])
#val_to = zip(post_tensors_data[1], comments_tensors_data[1])
val_data = RedditDataset(post_tensors_data[1], comments_tensors_data[1])
#test_to = zip(post_tensors_data[2], comments_tensors_data[2])
test_data = RedditDataset(post_tensors_data[2], comments_tensors_data[2])
#print(len(comments_tensors_data[0]))
#print(len(posts_datasets_to_use[0]))
#print(len(comments_datasets_to_use[0]))

In [ ]:
# model = GPT2LMHeadModel.from_pretrained('gpt2')
import torch
#from transformers import TextDataset
#print(len(train_dataset_to_use))
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))
#OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, "Cornell/checkpoints")
OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, "Ohio/checkpoints-Ohio")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    # do_train=True,
    num_train_epochs=1, # change this back to 2 
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy = "steps", #comment this
    eval_steps = 200,
    save_steps= 800,
    warmup_steps = 200,
    load_best_model_at_end = True, #comment this
    prediction_loss_only = True,
    )
# try and fix the above params as well
# convert dataset_to_use from a dictionary to a dataset type so that we can use it in the trainer
train_dataset_to_use_converted = torch.utils.data.DataLoader(train_dataset_to_use)
#print(type(train_dataset_to_use_converted))
test_dataset_to_use_converted = torch.utils.data.DataLoader(test_dataset_to_use)

#early_stop = EarlyStoppingCallback(5, 1.0) #comment this

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=train_data,
    eval_dataset=test_data
)

# depends on whether we want to use Trainer or not. I think using Trainer would be better if we get it to work,
# but I'm not sure how feasible it is. Additionally manual is kinda what we did in class
# ask about batching our data, and ask about num_labels. Batching our data in that dataloader part. like how to iterate over batches of data.
# idk if need to batch

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
200,No log,3.359580
400,No log,3.351166
600,3.400300,3.348058
800,3.400300,3.343915
1000,3.381400,3.341383
1200,3.381400,3.334601
1400,3.381400,3.338468
1600,3.318700,3.329915


In [ ]:
trainer.save_model()

VVVVVVVVrun if needed

In [ ]:
OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, "Ohioheckpoints-Ohio")
model = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token = "<bos>", eos_token ="<eos>", truncation_side='right')
3

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


^^^^^^run if needed 

In [ ]:
#https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets-demo.ipynb#scrollTo=ZSCf6QyF8AG-
post_in_one = "I saw a squirrel today. I feel blessed. So fluffy. I saw it eat cute nuts."
post_in = "I want to go to hotel management school for rural bed and breakfasts. What experiences or classes should I seek out?"
post_with_bos = "<bos>" + post_in + " TL;DR"
encoded_prompt = tokenizer(post_with_bos, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(model.device)

# prediction
output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_new_tokens=75,
    min_new_tokens=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=3
    )
generated_sequences = []
print(len(output_sequences))
torch.unique(output_sequences, dim=0)
print(len(output_sequences))
# decode prediction
for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    #if not ALLOW_NEW_LINES:
        #limit = text.find('\n')
        #text = text[: limit if limit != -1 else None]
    generated_sequences.append(text.strip())

In [ ]:
def make_sent(input):
    output = ""
    end_things = ".!?"
    last_end = 0
    i = 0
    while i < len(input):
        if input[i].isascii() or "'":
            if input[i] in end_things:
                output = output + input[i] + "\n"
                i = i + 1
                last_end = i
                if i + 1 < len(input):
                    if input[i] == " ":
                        i = i + 1
                        if i < len(input):
                            while input[i] == " ":
                                i = i + 1
                                if i >= len(input):
                                    break
                            output = output + " "  
            else:
                if i + 1 == len(input):
                    if input[i] == ",":
                        output = output + "."
                    else:
                        output = output + input[i]
                        output = output + "."
                else:
                    output = output + input[i]
                i = i + 1
    return output

In [ ]:
#https://python.plainenglish.io/use-python-to-split-sentences-into-separate-lines-17a577f9ddb3
print("Post: ")

post_out_now = make_sent(post_in)
print(post_out_now)

i = 1
for x in generated_sequences:
    out = x.split("TL;DR")
    comment = out[1]
    print("Comment {}: ".format(i))
    comment_out_now = make_sent(comment)
    print(comment_out_now)
    i = i+1
    print('\n')

Validation

In [ ]:
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=37005d7a35c019bbc61f80a9f86f9b734ac3022561a6e040032c016852cc14cd
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer
from torch import optim
from torch.utils.data import Dataset
from tqdm import tqdm

In [ ]:
# print(post_tensors_data[1])
# print(comments_datasets_to_use[1])
def validation_generate(post):
  # print(post)
  post_with_bos = "<bos>" + post + " TL;DR"
  encoded_prompt = tokenizer(post_with_bos, return_tensors="pt").input_ids
  encoded_prompt = encoded_prompt.to(model.device)
  # print(encoded_prompt)

  # prediction
  output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_new_tokens=75,
    min_new_tokens=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=1
  )
    
  generated_sequences = []
  # decode prediction
  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    generated_sequences.append(text.strip())
  
  # print(generated_sequences)
  return generated_sequences

In [ ]:
val_len = len(comments_datasets_to_use[1])
val_dataset = [{} for sub in range(val_len)]

for cnt, item in enumerate(comments_datasets_to_use[1]):
  val_dataset[cnt]['reference_summary'] = item

for cnt, post in enumerate(post_tensors_data[1]):
  # print(post)
  generated_sequences = validation_generate(post)

  for sent in generated_sequences:
    sent = sent.replace(post, '').replace('TL;DR', '')
    val_dataset[cnt]['generated_text'] = sent
    # print(sent)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

**Calculating the Rouge score**

In [ ]:
!pip3 install Rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
generated_text = val_dataset[7]['generated_text']
reference_summary = val_dataset[7]['reference_summary']
print("GENERATED COMMENTS: {}\nReference COMMENTS:{}".format(generated_text, reference_summary))

GENERATED COMMENTS: Hey IU....  The [IU Ski and Snowboard Club]([url link] is once again hosting our annual Rail Jam in Dunn Meadow. We're bring in real rails from Paoli Peaks, and real snow from the local ice rinks. We'll also have a DJ, redbull girls, and some other fun stuff going on. Come watch, come ride, join us! **Friday November 30th- 6pm to 9pm** Dunn Meadow- Right outside of the Indiana Memorial Union along 7th, Indiana Ave.  [MORE DETAILS HERE]([url link] All participants must wear a helmet and sign our waiver.  ----- The [IU Ski and Snowboard Club]([url link] in partnership with the Bloomington Alpine Club, and Paoli Peaks is premiering MSP's Superheros of Stoke on Sunday December 2nd. **Where:** Indiana Memorial Union-- Whittenberger Auditorium on the campus of Indiana University- Bloomington **When:** Doors open at 6PM **What:** We're premiering Superheros of Stoke along with Burtons 13. We will also be raffling off thousands of dollars in prizes including some goggles fr

In [ ]:
from rouge import Rouge

# create a Rouge object
rouge = Rouge()

# initialize scores for each variant of ROUGE
scores_rouge1 = {'recall': 0, 'precision': 0, 'fmeasure': 0}
scores_rouge2 = {'recall': 0, 'precision': 0, 'fmeasure': 0}
scores_rougeL = {'recall': 0, 'precision': 0, 'fmeasure': 0}

for idx in range(val_len):
    generated_text = val_dataset[idx]['generated_text']
    reference_summary = val_dataset[idx]['reference_summary']
    
    scores = rouge.get_scores(generated_text, reference_summary, avg=True)
    
    # compute ROUGE-1 scores
    scores_rouge1['recall'] += scores['rouge-1']['r']
    scores_rouge1['precision'] += scores['rouge-1']['p']
    scores_rouge1['fmeasure'] += scores['rouge-1']['f']
    
    # compute ROUGE-2 scores
    # scores = rouge.get_scores(generated_text, reference_summary, avg=True, rouge_n=(2, 2))
    scores_rouge2['recall'] += scores['rouge-2']['r']
    scores_rouge2['precision'] += scores['rouge-2']['p']
    scores_rouge2['fmeasure'] += scores['rouge-2']['f']
    
    # compute ROUGE-L scores
    # scores = rouge.get_scores(generated_text, reference_summary, avg=True, rouge_l=True)
    scores_rougeL['recall'] += scores['rouge-l']['r']
    scores_rougeL['precision'] += scores['rouge-l']['p']
    scores_rougeL['fmeasure'] += scores['rouge-l']['f']

# compute the average scores over all examples
n_examples = val_len
scores_rouge1 = {k: v/n_examples for k, v in scores_rouge1.items()}
scores_rouge2 = {k: v/n_examples for k, v in scores_rouge2.items()}
scores_rougeL = {k: v/n_examples for k, v in scores_rougeL.items()}

# print the overall scores
print("Overall ROUGE-1 scores:", scores_rouge1)
print("Overall ROUGE-2 scores:", scores_rouge2)
print("Overall ROUGE-L scores:", scores_rougeL)

Overall ROUGE-1 scores: {'recall': 0.23302455028359886, 'precision': 0.1956804989150321, 'fmeasure': 0.20125354200539675}
Overall ROUGE-2 scores: {'recall': 0.03019350859628833, 'precision': 0.02375709546853911, 'fmeasure': 0.024692817571042552}
Overall ROUGE-L scores: {'recall': 0.20652659513650415, 'precision': 0.17331510069622535, 'fmeasure': 0.17825576540250582}


Sentiment

In [ ]:
!pip3 install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 4.9 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
def find_label(scores):
  if scores['pos'] > scores['neg'] and scores['pos'] > scores['neu']:
    dominant_label = 'positive'
  elif scores['neg'] > scores['pos'] and scores['neg'] > scores['neu']:
    dominant_label = 'negative'
  else:
    dominant_label = 'neutral'
  
  return dominant_label

In [ ]:
acc = 0
sid = SentimentIntensityAnalyzer()
for idx in range(val_len):
  # print(val_dataset[idx]['reference_summary'])
  # print(type(val_dataset[idx]['reference_summary']))
  sent_gt = sid.polarity_scores(val_dataset[idx]['reference_summary'])
  sent_generated_comment = sid.polarity_scores(val_dataset[idx]['generated_text'])

  gt_label = find_label(sent_gt)
  generated_comment_label = find_label(sent_generated_comment)

  if gt_label == generated_comment_label:
    acc += 1

  print("Reference comments: {}".format(val_dataset[idx]['reference_summary']))
  print(sent_gt)
  print(gt_label)
  print("Generated comments: {}".format(val_dataset[idx]['generated_text']))
  print(sent_generated_comment)
  print(generated_comment_label)
  if idx == 15:
    break

print("The number of generated comments that have aligned sentiments: {}".format(acc))
print("The number of posts in the validation set: {}".format(val_len))
print("The accuracy of sentiments of the generated comments: {}".format(acc / val_len))

Reference comments: <bos> I just got back from mine that was today. Pretty much the entire day is sitting in a room listening to people tell you stuff that can be found online. There was a live student musical about rape, free drawstring IU bags with sunglasses and pens, and most importantly, an ID name tag that gave you your appointment time for the next day with your advisor. If you really can't bare to stay, at least go get your badge with your appointment time. I don't know how you would know when to go the second day without it. <eos> 
{'neg': 0.044, 'neu': 0.87, 'pos': 0.086, 'compound': 0.5256}
neutral
Generated comments: I am an intercampus transfer and there is orientation tomorrow and the day after. It's 2 days long. I don't feel like attending that long of an orientation, most other schools have orientations a day long especially for transfer students. Can I skip most of the orientation and just have the advising appointment? Or will that not work and they will place a hold 

In [ ]:
for item in comments_datasets_to_use[1]:
  print(item)

<bos> I just got back from mine that was today. Pretty much the entire day is sitting in a room listening to people tell you stuff that can be found online. There was a live student musical about rape, free drawstring IU bags with sunglasses and pens, and most importantly, an ID name tag that gave you your appointment time for the next day with your advisor. If you really can't bare to stay, at least go get your badge with your appointment time. I don't know how you would know when to go the second day without it. <eos> 
<bos> There's a list at Broke Student's Guide, couple years old but should still be mostly applicable. [url link] Not free but I've heard you can put raybans on your bursar at the optometry building. Can anyone confirm this? <eos> 
<bos> They can (and will) send it to collections. They will send something to the address that the car is registered to if its not paid. At least that's what happened with my parking ticket last year. Just appeal the damn thing, it tells you